Configuración Inicial

In [21]:
from snowflake.snowpark import Session

creds = {
  "account"   : "avb19033",
  "user"      : "andresmitre",
  "password"  : "LbnK_t:Fl)&RD5",
  "role"      : "ACCOUNTADMIN"
}

session = Session.builder.configs(creds).create()

c:\Users\andres.mitre\AppData\Local\Programs\Python\Python311\Lib\site-packages\snowflake\connector\config_manager.py:351: UserWarning: Bad owner or permissions on C:\Users\andres.mitre\.snowflake\connections.toml
  warn(f"Bad owner or permissions on {str(filep)}{chmod_message}")


In [23]:
session.sql("SHOW SCHEMAS").show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"created_on"                      |"name"              |"is_default"  |"is_current"  |"database_name"  |"owner"       |"comment"                                           |"options"  |"retention_time"  |"owner_role_type"  |"budget"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2024-08-22 14:48:16.606000-07:00  |INFORMATION_SCHEMA  |N             |N             |ANALYTICAL       |              |Views describing the contents of schemas in thi...  |           |1                 |                   |NULL      |
|2024-08-21 09:12:19.591000-07:00  |PUBLIC              

Crear tablas en nueva base de datos

In [24]:
# Crear una nueva base de datos
session.sql("CREATE OR REPLACE DATABASE REGRESSION_DB").collect()

# Crear un esquema dentro de la base de datos
session.sql("CREATE OR REPLACE SCHEMA REGRESSION_DB.PUBLIC").collect()

# Crear la tabla de datos de ventas y publicidad
create_data_table = """
CREATE OR REPLACE TABLE REGRESSION_DB.PUBLIC.SALES_ADVERTISING (
    ID INT,
    ADVERTISING_EXPENSE DECIMAL(10, 2),
    SALES DECIMAL(10, 2)
);
"""
session.sql(create_data_table).collect()

[Row(status='Table SALES_ADVERTISING successfully created.')]

Insertar datos

In [25]:
insert_data_query = """
INSERT INTO REGRESSION_DB.PUBLIC.SALES_ADVERTISING (ID, ADVERTISING_EXPENSE, SALES) VALUES
(1, 100.00, 200.00),
(2, 150.00, 250.00),
(3, 200.00, 300.00),
(4, 250.00, 350.00),
(5, 300.00, 400.00);
"""
session.sql(insert_data_query).collect()

[Row(number of rows inserted=5)]

Crear vista

In [ ]:
create_view_query = """
CREATE OR REPLACE VIEW REGRESSION_DB.PUBLIC.SALES_ADVERTISING_VIEW AS
SELECT * FROM REGRESSION_DB.PUBLIC.SALES_ADVERTISING;
"""
session.sql(create_view_query).collect()

In [15]:
create_task_1 = """
CREATE OR REPLACE TASK REGRESSION_DB.PUBLIC.TASK_INSERT_DATA_2
  WAREHOUSE = DEVELOPER
  SCHEDULE = 'USING CRON 0 * * * * UTC'
AS
  INSERT INTO REGRESSION_DB.PUBLIC.SALES_ADVERTISING (ID, ADVERTISING_EXPENSE, SALES)
  VALUES (6, 350.00, 450.00);
"""
session.sql(create_task_1).collect()

[Row(status='Task TASK_INSERT_DATA_2 successfully created.')]

Tarea 2: Actualizar un registro existente

In [16]:
create_task_2 = """
CREATE OR REPLACE TASK REGRESSION_DB.PUBLIC.TASK_UPDATE_DATA_2
  WAREHOUSE = DEVELOPER
  AFTER REGRESSION_DB.PUBLIC.TASK_INSERT_DATA_2
AS
  UPDATE REGRESSION_DB.PUBLIC.SALES_ADVERTISING
  SET SALES = 460.00
  WHERE ID = 1;
"""
session.sql(create_task_2).collect()

[Row(status='Task TASK_UPDATE_DATA_2 successfully created.')]

Tarea 3: Eliminar un registro

In [17]:
create_task_3 = """
CREATE OR REPLACE TASK REGRESSION_DB.PUBLIC.TASK_DELETE_DATA_2
  WAREHOUSE = DEVELOPER
  AFTER REGRESSION_DB.PUBLIC.TASK_UPDATE_DATA_2
AS
  DELETE FROM REGRESSION_DB.PUBLIC.SALES_ADVERTISING
  WHERE ID = 6;
"""
session.sql(create_task_3).collect()


[Row(status='Task TASK_DELETE_DATA_2 successfully created.')]

In [20]:
session.sql("ALTER TASK REGRESSION_DB.PUBLIC.TASK_INSERT_DATA_2 RESUME").collect()
session.sql("ALTER TASK REGRESSION_DB.PUBLIC.TASK_INSERT_DATA_2 RESUME").collect()
session.sql("ALTER TASK REGRESSION_DB.PUBLIC.TASK_INSERT_DATA_2 RESUME").collect()

[Row(status='Statement executed successfully.')]

Verificar DAG

In [19]:
session.sql("SHOW TASKS LIKE 'REGRESSION_DB.PUBLIC.TASK_UPDATE_DATA_2'")

In [6]:
# Verificar la vista
session.sql("SELECT * FROM REGRESSION_DB.PUBLIC.SALES_ADVERTISING_VIEW").show()

# Verificar la tabla
session.sql("SELECT * FROM REGRESSION_DB.PUBLIC.SALES_ADVERTISING").show()

# Verificar las tareas
session.sql("SHOW TASKS LIKE 'REGRESSION_DB.PUBLIC.TASK_%'").show()


SnowparkSQLException: (1304): 01b68673-0002-cff5-0000-27d3000280d6: 002003 (42S02): SQL compilation error:
Object 'REGRESSION_DB.PUBLIC.SALES_ADVERTISING_VIEW' does not exist or not authorized.

In [16]:
session.sql("SELECT * FROM GARDEN_PLANTS.VEGGIES.ROOT_DEPTH WHERE ROOT_DEPTH_ID = 3").show()

-----------------------------------------------------------------------------------------------------------
|"ROOT_DEPTH_ID"  |"ROOT_DEPTH_CODE"  |"ROOT_DEPTH_NAME"  |"UNIT_OF_MEASURE"  |"RANGE_MIN"  |"RANGE_MAX"  |
-----------------------------------------------------------------------------------------------------------
|3                |L                  |Large              |cm                 |60           |90           |
-----------------------------------------------------------------------------------------------------------



In [21]:
session.sql("""
    INSERT INTO GARDEN_PLANTS.VEGGIES.ROOT_DEPTH 
    (ROOT_DEPTH_ID, ROOT_DEPTH_CODE, ROOT_DEPTH_NAME, UNIT_OF_MEASURE, RANGE_MIN, RANGE_MAX) 
    VALUES 
    (6, 'X', 'Extra', 'cm', 95, 95)
""").collect()


[Row(number of rows inserted=1)]

In [22]:
session.sql("CREATE SCHEMA my_new_schema").collect()

SnowparkSQLException: (1304): 01b6822c-0002-cab5-0000-27d30001c3ba: 090105 (22000): Cannot perform CREATE SCHEMA. This session does not have a current database. Call 'USE DATABASE', or use a qualified name.